In [1]:
!apt-get update -qq
!apt-get install -y -qq poppler-utils tesseract-ocr

!pip install -q sentence-transformers transformers pdfplumber PyMuPDF pytesseract pillow gradio

!pip install -q spacy
!python -m spacy download en_core_web_sm

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package poppler-utils.
(Reading database ... 126718 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.11_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.11) ...
Setting up poppler-utils (22.02.0-2ubuntu0.11) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 959.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.6 MB/s eta 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os, re, tempfile, time
from pathlib import Path
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import pdfplumber, fitz, pytesseract
from PIL import Image
import gradio as gr


MODEL_DIR = "/content/drive/MyDrive/resume_qa_Model"
DATA_DIR = "/content/drive/MyDrive/resume_data"
print("MODEL_DIR:", MODEL_DIR)
print("GPU available:", torch.cuda.is_available())
DEVICE = 0 if torch.cuda.is_available() else -1


MODEL_DIR: /content/drive/MyDrive/resume_qa_Model
GPU available: False


In [4]:
!ls -la "/content/drive/MyDrive/resume_qa_Model"


total 489329
drwx------ 2 root root      4096 Oct 28 18:16 checkpoint-228
-rw------- 1 root root       721 Oct 28 18:16 config.json
-rw------- 1 root root    456318 Oct 28 18:17 merges.txt
-rw------- 1 root root 496250232 Oct 28 18:16 model.safetensors
-rw------- 1 root root       957 Oct 28 18:17 special_tokens_map.json
-rw------- 1 root root      1302 Oct 28 18:17 tokenizer_config.json
-rw------- 1 root root   3558907 Oct 28 18:17 tokenizer.json
-rw------- 1 root root    798293 Oct 28 18:17 vocab.json


In [5]:
# Load embedding retriever
print("Loading embedding model (all-mpnet-base-v2)...")
retriever = SentenceTransformer("all-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained QA (fallback)
print("Loading pretrained QA model (deepset/roberta-base-squad2)...")
pretrained_name = "deepset/roberta-base-squad2"
pre_tokenizer = AutoTokenizer.from_pretrained(pretrained_name)
pre_model = AutoModelForQuestionAnswering.from_pretrained(pretrained_name)
qa_pretrained = pipeline("question-answering", model=pre_model, tokenizer=pre_tokenizer, device=DEVICE)

# Load fine-tuned QA from Drive (local files)
print("Loading fine-tuned model from Drive...")
ft_tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)
ft_model = AutoModelForQuestionAnswering.from_pretrained(MODEL_DIR, local_files_only=True)
qa_finetuned = pipeline("question-answering", model=ft_model, tokenizer=ft_tokenizer, device=DEVICE)

print("Models loaded.")


Loading embedding model (all-mpnet-base-v2)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

Device set to use cpu


Loading fine-tuned model from Drive...


Device set to use cpu


Models loaded.


In [6]:
def clean_text(t: str) -> str:
    if not t:
        return ""
    t = re.sub(r'\r\n?', '\n', t)
    t = re.sub(r'\n{2,}', '\n\n', t)
    t = re.sub(r'[ \t]+', ' ', t)
    return t.strip()

def extract_text_from_pdf(pdf_path: str) -> str:
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for p in pdf.pages:
                if p.extract_text():
                    text += p.extract_text() + "\n"
    except Exception:
        pass
    if len(text.split()) < 50:
        try:
            doc = fitz.open(pdf_path)
            text = ""
            for p in doc:
                text += p.get_text("text") + "\n"
        except Exception:
            pass
    if len(text.split()) < 50:
        try:
            doc = fitz.open(pdf_path)
            ocr_t = ""
            for p in doc:
                pix = p.get_pixmap()
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                ocr_t += pytesseract.image_to_string(img) + "\n"
            text = ocr_t
        except Exception:
            pass
    return clean_text(text)


In [7]:
def chunk_text(text: str, max_words=400, overlap=75):
    if not text:
        return []
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words - overlap):
        chunk = " ".join(words[i:i+max_words])
        if len(chunk.split())>5:
            chunks.append(chunk)
    return chunks


In [8]:
import numpy as np

def get_top_k_chunks(question: str, chunks: list, top_k=5):
    if not chunks:
        return []
    q_emb = retriever.encode(question, convert_to_tensor=True)
    chunk_embs = retriever.encode(chunks, convert_to_tensor=True)
    sims = util.cos_sim(q_emb, chunk_embs)[0].cpu().numpy()
    top_idx = np.argsort(-sims)[:min(top_k, len(chunks))]
    return [chunks[i] for i in top_idx], [float(sims[i]) for i in top_idx]


In [12]:
def run_hybrid_qa(question: str, text: str, top_k=5, combine_n=3):
    """
    1) chunk text
    2) retrieve top_k chunks
    3) run both models on each chunk (and combined top-N)
    4) combine scores and rerank answers, deduplicate
    """
    chunks = chunk_text(text)
    if not chunks:
        return []
    top_chunks, sims = get_top_k_chunks(question, chunks, top_k=top_k)

    candidates = []
    # run on individual chunks
    for i, chunk in enumerate(top_chunks):
        #  trimmed for speed (pipelines truncate automatically)
        try:
            res_ft = qa_finetuned(question=question, context=chunk)
            ans_ft = res_ft.get("answer","").strip()
            score_ft = float(res_ft.get("score", 0.0))
        except Exception:
            ans_ft = ""; score_ft = 0.0
        try:
            res_pre = qa_pretrained(question=question, context=chunk)
            ans_pre = res_pre.get("answer","").strip()
            score_pre = float(res_pre.get("score", 0.0))
        except Exception:
            ans_pre = ""; score_pre = 0.0

        # collect if non-empty
        if ans_ft:
            candidates.append({"answer":ans_ft, "model":"fine", "model_score":score_ft, "retrieval_score":sims[i]})
        if ans_pre:
            candidates.append({"answer":ans_pre, "model":"pre", "model_score":score_pre, "retrieval_score":sims[i]})


    if combine_n > 1 and len(top_chunks) >= 1:
        combined_ctx = "\n".join(top_chunks[:combine_n])
        try:
            res_ftc = qa_finetuned(question=question, context=combined_ctx)
            if res_ftc.get("answer"):
                candidates.append({"answer":res_ftc["answer"].strip(),"model":"fine_comb","model_score":float(res_ftc.get("score",0.0)),"retrieval_score":float(np.mean(sims))})
        except Exception:
            pass
        try:
            res_prec = qa_pretrained(question=question, context=combined_ctx)
            if res_prec.get("answer"):
                candidates.append({"answer":res_prec["answer"].strip(),"model":"pre_comb","model_score":float(res_prec.get("score",0.0)),"retrieval_score":float(np.mean(sims))})
        except Exception:
            pass

    # Merge duplicates (case-insensitive) keeping highest composite score
    merged = {}
    for c in candidates:
        key = c["answer"].strip().lower()
        # composite score = weighted model_score + retrieval_score (tunable)
        weight_model = 0.85 if c["model"].startswith("fine") else 0.7
        composite = weight_model * c["model_score"] + 0.15 * c["retrieval_score"]
        if key in merged:
            if composite > merged[key]["composite"]:
                merged[key] = {"answer": c["answer"], "composite": composite, "model": c["model"], "model_score": c["model_score"]}
        else:
            merged[key] = {"answer": c["answer"], "composite": composite, "model": c["model"], "model_score": c["model_score"]}

    # sort results
    final = sorted(merged.values(), key=lambda x: x["composite"], reverse=True)
    return final


In [13]:

sample_pdf = sorted(list(Path(DATA_DIR).glob("*.pdf")))[0] if Path(DATA_DIR).exists() else None
if sample_pdf:
    text = extract_text_from_pdf(str(sample_pdf))
    q = "What is the candidate's email address?"
    results = run_hybrid_qa(q, text, top_k=5, combine_n=3)
    print("Top results:", results[:3])
else:
    print("No sample PDF found in DATA_DIR. Upload at:", DATA_DIR)


Top results: [{'answer': 'aniruddha.sharma0019@gmail.com', 'composite': 1.7101899822025641, 'model': 'fine_comb', 'model_score': 1.9980545060125223}, {'answer': 'ANIRUDDHA SHARMA', 'composite': 0.0533122851891676, 'model': 'pre', 'model_score': 0.060913532361155376}, {'answer': '.', 'composite': 0.010672812568551807, 'model': 'fine', 'model_score': 3.787408646253876e-11}]


In [14]:
# Gradio functions
def answer_from_uploaded(pdf_file, question):
    # pdf_file.name is the path to the temporary uploaded file
    pdf_path = pdf_file.name if hasattr(pdf_file, "name") else None
    if not pdf_path or not Path(pdf_path).exists():

        tf = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
        tf.write(pdf_file.read())
        tf.flush()
        pdf_path = tf.name

    text = extract_text_from_pdf(pdf_path)
    out = run_hybrid_qa(question, text, top_k=5, combine_n=3)
    if not out:

        emails = re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', text)
        phones = re.findall(r'(\+?\d[\d\-\s\(\)]{7,}\d)', text)
        names = re.findall(r'([A-Z][a-z]+(?: [A-Z][a-z]+)+)', text[:200])
        fallback = []
        if emails: fallback.append({"answer":emails[0],"composite":1.0})
        if phones: fallback.append({"answer":phones[0],"composite":1.0})
        if names: fallback.append({"answer":names[0],"composite":0.95})
        if fallback:
            return f"Fallback results:\n" + "\n".join([f"{r['answer']} (score {r['composite']})" for r in fallback])
        return "No good answer found."

    # return top 3 nicely formatted
    out_lines = [f"1) {out[0]['answer']}  (score {out[0]['composite']:.3f})"]
    for i, r in enumerate(out[1:3], start=2):
        out_lines.append(f"{i}) {r['answer']}  (score {r['composite']:.3f})")
    return "\n".join(out_lines)

# Gradio UI components
with gr.Blocks() as demo:
    gr.Markdown("## Resume Question Answering (Hybrid: fine-tuned + pretrained)")
    with gr.Row():
        pdf_in = gr.File(label="Upload Resume PDF", file_types=[".pdf"])
        question = gr.Textbox(label="Your question", placeholder="What is the candidate's email address?")
    output = gr.Textbox(label="Answer")
    btn = gr.Button("Get Answer")
    btn.click(answer_from_uploaded, inputs=[pdf_in, question], outputs=output)

demo.launch(debug=False, share=True, inline=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8c78f441dc62d14c6b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
